In [15]:
!pip install pyspark

In [16]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f

In [18]:
spark

In [21]:
Crime_data = 'Part1_Crime_data.csv'

In [22]:
crime_schema = StructType([StructField('X', DoubleType(), True),
StructField('Y', DoubleType(), True),
StructField('RowID', StringType(), True),
StructField('CrimeDateTime', StringType(), True),
StructField('CrimeCode', StringType(), True),
StructField('Location', StringType(), True),
StructField('Description', StringType(), True),
StructField('Inside_Outside', StringType(), True),
StructField('Weapon', StringType(), True),
StructField('Post', StringType(), True),
StructField('District', StringType(), True),
StructField('Neighborhood', StringType(), True),
StructField('Latitude', DoubleType(), True),
StructField('Longitude', DoubleType(), True),
StructField('GeoLocation', StringType(), True),
StructField('Premise', StringType(), True),
StructField('VRIName', StringType(), True),
StructField('Total_Incidents', IntegerType(), True),
StructField('Shape', StringType(), True)])

Reading CSV file with the structured schema

In [23]:
df = spark.read.csv(Crime_data, header=True, schema=crime_schema)

Cache the Dataframe

In [24]:
df.cache()

DataFrame[X: double, Y: double, RowID: string, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: string, District: string, Neighborhood: string, Latitude: double, Longitude: double, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int, Shape: string]

Total number of rows

In [25]:
df.count()

513158

Print the schema

In [26]:
df.printSchema()

root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- RowID: string (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



Top 5 rows of the dataframe

In [28]:
df.head(5)

[Row(X=1420074.13302107, Y=594160.602354662, RowID='1', CrimeDateTime='2022/02/26 04:00:00+00', CrimeCode='4E', Location='200 W MONUMENT ST', Description='COMMON ASSAULT', Inside_Outside=None, Weapon='NA', Post='124', District='CENTRAL', Neighborhood='MOUNT VERNON', Latitude=39.2975, Longitude=-76.6193, GeoLocation='(39.2975,-76.6193)', Premise=None, VRIName=None, Total_Incidents=1, Shape=None),
 Row(X=1411374.22509631, Y=589791.383964529, RowID='2', CrimeDateTime='2022/02/26 01:26:30+00', CrimeCode='9S', Location='2100 FREDERICK AVE', Description='SHOOTING', Inside_Outside='Outside', Weapon='FIREARM', Post='835', District='SOUTHWEST', Neighborhood='CARROLLTON RIDGE', Latitude=39.2856, Longitude=-76.6501, GeoLocation='(39.2856,-76.6501)', Premise='COMMON BUSINESS', VRIName='Tri-District', Total_Incidents=1, Shape=None),
 Row(X=1411401.16887136, Y=582761.775193539, RowID='3', CrimeDateTime='2022/02/26 10:22:00+00', CrimeCode='4E', Location='2000 GRINNALDS AVE', Description='COMMON ASSAU

**Distinct Crime codes**

In [35]:
#Number of distinct crime codes
df.createOrReplaceTempView('DistinctCrimecode')

query = """
SELECT COUNT(DISTINCT Crimecode) AS Count
FROM DistinctCrimecode 

"""
spark.sql(query).show()

+-----+
|Count|
+-----+
|   85|
+-----+



In [36]:
#Distinct crime codes
df.createOrReplaceTempView('DistinctCrimecode')

query = """
SELECT DISTINCT Crimecode
FROM DistinctCrimecode

"""
spark.sql(query).show(85)

+---------+
|Crimecode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|       6K|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8GV|
|      8AO|
|       7C|
|      3AK|
|      3GK|
|       6L|
|      3EO|
|      3JO|
|       3F|
|       1K|
|       8H|
|      8CV|
|      8DO|
|       4C|
|       5A|
|       6C|
|      3NK|
|       3D|
|       6H|
|      3LK|
|     3AJK|
|      3CO|
|       3L|
|       4E|
|      8BV|
|       6D|
|       2A|
|       3C|
|       8I|
|      3NO|
|      3JF|
|       3E|
|      3LF|
|       1O|
|       8J|
|      3CK|
|      8BO|
|       2B|
|      3JK|
|       5B|
|       4A|
|      8GO|
|      8EV|
|      3CF|
|      8EO|
|       6G|
|       6A|
|       9S|
|      3EK|
|      8FV|
|      3GF|
|      8CO|
|       3H|
|       4D|
|      8FO|
|       6J|
|       6F|
|       6E|
|       3J|
|       5C|
|       5G|
|   

**Number of crimes per crime code in descending order**

In [38]:
df.createOrReplaceTempView('Numberofcrimes')

query = """
SELECT Crimecode, SUM(Total_Incidents) AS Number_of_Crimes
FROM Numberofcrimes
GROUP BY Crimecode
ORDER BY SUM(Total_Incidents) DESC

"""
spark.sql(query).show()

+---------+----------------+
|Crimecode|Number_of_Crimes|
+---------+----------------+
|       4E|           91650|
|       6D|           68427|
|       5A|           43928|
|       7A|           40274|
|       6J|           27636|
|       6G|           26858|
|       6E|           24300|
|       6C|           23227|
|       4C|           22438|
|       5D|           14971|
|      3AF|           14739|
|       4B|           14460|
|       4A|           13226|
|       3B|           10737|
|       4D|            7232|
|       5B|            6475|
|       9S|            5443|
|       6F|            5081|
|       5C|            4917|
|       6B|            4106|
+---------+----------------+
only showing top 20 rows



**Neighborhood with highest crimes**

In [49]:
df.createOrReplaceTempView('NeighborhoodCrimes')

query = """

SELECT Neighborhood AS Highest_Crime_City, SUM(Total_Incidents) AS Crime_count FROM NeighborhoodCrimes 
GROUP BY Neighborhood 
ORDER BY SUM(Total_Incidents) DESC

"""
spark.sql(query).show(1)

+------------------+-----------+
|Highest_Crime_City|Crime_count|
+------------------+-----------+
|          DOWNTOWN|      17799|
+------------------+-----------+
only showing top 1 row



**Month having most crimes**

In [43]:
df.createOrReplaceTempView('CrimeMonth')

query = """

SELECT CAST(SUBSTRING(CrimeDateTime, 6, 2) AS int)  AS Highest_Crime_Month, SUM(Total_Incidents) AS Count FROM CrimeMonth
GROUP BY Highest_Crime_Month
ORDER BY SUM(Total_Incidents) DESC

"""
spark.sql(query).show(1)

+-------------------+-----+
|Highest_Crime_Month|Count|
+-------------------+-----+
|                  8|46327|
+-------------------+-----+
only showing top 1 row



**Weapons Used**

In [14]:
df.createOrReplaceTempView('DistinctWeapons')

query = """

SELECT DISTINCT Weapon FROM DistinctWeapons
WHERE Weapon <> 'NA' 
"""
spark.sql(query).show()

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



**Most used Weapon**

In [13]:
df.createOrReplaceTempView('WeaponUsage')

query = """

SELECT Weapon , SUM(Total_Incidents) AS Total_Usage FROM WeaponUsage 
WHERE Weapon <> 'NA'
GROUP BY Weapon 
ORDER BY SUM(Total_Incidents) DESC

"""
spark.sql(query).show(1)

+-------+-----------+
| Weapon|Total_Usage|
+-------+-----------+
|FIREARM|      46139|
+-------+-----------+
only showing top 1 row

